# datastore

> Datastore for obsidian notes

In [ ]:
#| default_exp datastore

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:

#| export
from dataclasses import dataclass
from typing import List, Dict, Optional
from fastlite import Database, diagram
from memexplatform_obsidian.commons import config, ExtensionTypes
import pathlib
from pathlib import Path
from datetime import datetime, timezone
from memexplatform_obsidian.mdmanager import ObsidianPage,  Link, RawText, AnyLink, TagLink, get_subdirs, resolve_note_path
import uuid
from memexplatform_obsidian.commons import MountPaths, ResponseTypes
from abc import ABC, ABCMeta, abstractmethod
import urllib
from memexplatform_obsidian.jupyter import JupyterPage
from fasthtml.common import *
from monsterui.all import *

In [ ]:
#| export
@dataclass
class Node:
    lockey: str
    created_time: float
    modified_time: float
    file_size: int
    fname: str
    parent: str
    text: str
    blob: bytes
    ext: str
    is_folder: bool
    title: str
    url: str
    obsidian_url: Optional[str]
    checksum: bytes

@dataclass
class Links:
    id: str
    lockey: str
    link: str
    linked_lockey: str
    link_type: str
    title: str
    is_internal: bool

@dataclass
class Properties:
    id: str
    lockey: str
    name: str
    type: str
    value: str


In [ ]:
#| export
def update_node(page, file_path):
    node_text_content = ""
    node_blob_content = b""
    if file_path.is_file():
        node_text_content = page.text
        node_blob_content = page._fpath.read_bytes()
    
    # Get stat info if not already populated by ObsidianPage (due to its internal logic
    # where created_time, modified_time, file_size are set to None for regular files).
    current_created_time = page.created_time
    current_modified_time = page.modified_time
    current_file_size = page.file_size
    # print(node_text_content, node_blob_content)
    # print(node_text_content)
    if page._fpath.is_file():
        try:
            s = page._fpath.stat()
            if current_created_time is None:
                current_created_time = datetime.fromtimestamp(s.st_ctime_ns / 1e9, tz=timezone.utc)
            if current_modified_time is None:
                current_modified_time = datetime.fromtimestamp(s.st_mtime_ns / 1e9, tz=timezone.utc)
            if current_file_size is None:
                current_file_size = s.st_size
        except Exception as e:
            print(f"Warning: Could not get stat info for {file_path}: {e}")
            pass # Keep as None or default if stat fails

    # Convert datetime objects to float timestamps, providing default 0.0 if None
    created_time_ts = current_created_time.timestamp() if current_created_time else 0.0
    modified_time_ts = current_modified_time.timestamp() if current_modified_time else 0.0
    file_size_val = current_file_size if current_file_size is not None else 0

    # Ensure extension is a string, default to empty string if None
    node_ext = page.file_extension if page.file_extension else ""

    # Construct the Node object
    node = Node(
        lockey=page.lockey,
        created_time=created_time_ts,
        modified_time=modified_time_ts,
        file_size=file_size_val,
        fname=file_path.name,
        parent=str(page.parent),
        title = page.title,
        text=node_text_content,
        blob=node_blob_content,
        ext=node_ext,
        is_folder=file_path.is_dir(), # True if it's a directory, False otherwise
        url=page.app_url,
        obsidian_url=str(page.obsidian_url) if page.obsidian_url else None,
        checksum=page.checksum
    )

    return node

In [ ]:
# resolve_note_path?

In [ ]:
#| export
def get_properties(page):
    fm = page.frontmatter
    plist = []
    if fm:
        for p in fm.children:
            for c in p.children:
                if isinstance(c,RawText):
                    if c.content:
                        plist.append(Properties(
                            id=str(uuid.uuid4()),
                            lockey=page.lockey,
                            name=p.key,
                            type=type(c).__name__,
                            value=c.content
                        ))
                elif isinstance(c,(Link, AnyLink)):
                    name = p.key
                    if isinstance(c, TagLink): name = "file_tags"
                    if getattr(c, 'fname', None):
                        plist.append(Properties(
                            id=str(uuid.uuid4()),
                            lockey=page.lockey,
                            name=name,
                            type=type(c).__name__,
                            value=getattr(c, 'fname')
                        ))

                    else:
                        plist.append(Properties(
                            id=str(uuid.uuid4()),
                            lockey=page.lockey,
                            name=name,
                            type=type(c).__name__,
                            value=getattr(c, 'target')
                        ))

    for c in page.tags:
        plist.append(Properties(
            id=str(uuid.uuid4()),
            lockey=page.lockey,
            name='tags',
            type=type(c).__name__,
            value=c.title
        ))

    return plist

In [ ]:

fpath = "/Users/rahul1.saraf/rahuketu/programming/notesobs/pages/@RahulSaraf.md"
fpath = "/Users/rahul1.saraf/rahuketu/programming/notesobs/Clippings/Fuck You, Show Me The Prompt. – 2.md"
# fpath = "/Users/rahul1.saraf/rahuketu/programming/notesobs/pages/Product Mindset for RAG.md"
op = ObsidianPage.from_file_path(fpath)
link = op.links[0]; link
# link.fname
vault = config.OBSIDIAN_VAULT
linked_path = resolve_note_path(vault, link.fname) if getattr(link, "fname", None) else None
linked_lockey = str(linked_path.relative_to(vault)) if linked_path else None; linked_lockey
op.lockey

link.target, link.title
fm = op.frontmatter
for p in fm.children:
    print(p.key, p.children)
op = ObsidianPage.from_file_path(fpath)
# get_properties(op)
# op.tags

title [<mistletoe.span_token.RawText content='Fuck You, Show Me The Prompt. '...+1>]
source [<memexplatform_obsidian.mdmanager.AnyLink with 1 child title='https://hamel.dev/blog/posts/p'...+6 target='https://hamel.dev/blog/posts/p'...+6 children=(<mistletoe.span_token.RawText content='https://hamel.dev/blog/posts/p'...+6>,)>]
author [<memexplatform_obsidian.mdmanager.WikiLink with 1 child ext='.md' fname='@HamelHusain' children=[<mistletoe.span_token.RawText content='@HamelHusain'>] title='@HamelHusain' label=None dest_type='wikilink' title_delimiter=None label=None src='/obsidian/open?file=%40HamelHu'...+4>]
published [<mistletoe.span_token.RawText content='None'>]
created [<mistletoe.span_token.RawText content='2025-08-30'>]
description [<mistletoe.span_token.RawText content='Quickly understand inscrutable'...+42>]
tags [<memexplatform_obsidian.mdmanager.TagLink with 1 child fname='#clippings' children=[<mistletoe.span_token.RawText content='#clippings'>] title='#clippings' label=Non

In [ ]:
#| export
def get_pagelinks(page:ObsidianPage, vault:Path):
    link_rows = []
    for link in page.links:
        # Try to resolve linked file path inside vault
        linked_path = resolve_note_path(vault, link.fname) if getattr(link, "fname", None) else None
        linked_lockey = str(linked_path.relative_to(vault)) if linked_path else None; linked_lockey

        link_rows.append(
            Links(
                id=str(uuid.uuid4()),
                lockey=page.lockey,       # source file
                link=link.target,           # raw link text
                linked_lockey=linked_lockey,
                link_type=type(link).__name__,
                title=link.title,
                is_internal=bool(linked_lockey or type(link).__name__ == 'TagLink'),
            )
        )
    return link_rows

In [ ]:
#| export
def update_folder_node(folder_path: Path, vault:Path) -> Node:
    """Create a Node entry for a folder."""
    try:
        s = folder_path.stat()
        created_time = datetime.fromtimestamp(s.st_ctime_ns / 1e9, tz=timezone.utc).timestamp()
        modified_time = datetime.fromtimestamp(s.st_mtime_ns / 1e9, tz=timezone.utc).timestamp()
    except Exception:
        created_time, modified_time = 0.0, 0.0

    return Node(
        lockey=str(folder_path.relative_to(vault)),
        created_time=created_time,
        modified_time=modified_time,
        file_size=0,
        fname=folder_path.name,
        parent=str(folder_path.parent.relative_to(vault)),
        title = folder_path.name,
        text="",
        blob=b"",
        ext="",
        is_folder=True,
        url=MountPaths.open.to(file=folder_path.relative_to(vault)),
        obsidian_url=None,
        checksum=b"",
    )

In [ ]:
#| export
def iter_files(dirs):
    for d in dirs:
        yield from (f for f in d.rglob("*") if f.is_file())


In [ ]:
#| export
@dataclass
class Backlink:
    lockey_page: str
    lockey_backlink: str
    db: Database

    @property
    def url(self):
        return MountPaths.open.to(file=self.lockey_backlink)


    @property
    def title(self):
        print(self.lockey_backlink)
        # return self.url
        return list(self.db['node'].rows_where("lockey = ?", (str(self.lockey_backlink),), select='title'))[0]['title']



    # def __ft__(self):
    #     return A(self.title, href=self.url)

In [ ]:
bl = Backlink(lockey_page='pages/RAG.md', lockey_backlink='pages/LLMCourse.md', db=Database(config.OBSIDIAN_DB))
bl.title, bl.url
# bl.__ft__()
# bl.url

# bl.db['node'].rows_where

pages/LLMCourse.md


('LLMCourse', '/obsidian/open?file=pages/LLMCourse.md')

In [ ]:
#| export
class NoteStore(ABC):
    @property
    @abstractmethod
    def subdirs(self):
        ...

    @abstractmethod
    def listing(self, folder): 
        ...
        
    @abstractmethod
    def query_file(self, file)->(Path,bool):
        ...


    @abstractmethod
    def backlinks(self, lockey):
        ...

In [ ]:
#| export
class FileStore(NoteStore):

    def __init__(self, config):
        self.config = config
        self.vault = config.OBSIDIAN_VAULT

    @property
    def subdirs(self)->List[Path|str]:
        subdirs = []
        for p in self.vault.rglob("*"):
            if p.is_dir():
                # Check if any parent directory (including self) should be skipped
                if any(part.startswith((".", "_")) or part.startswith("logseq") for part in p.parts): continue
                subdirs.append(p)
        return subdirs

    def listing(self, folder):
        for f in iter_files([folder]):
            name = f.name
            rel_path = f.relative_to(self.vault)
            href = MountPaths.open.to(file=rel_path)
            yield {'fname': name, 'url': href}

    def query_file(self, file) -> Dict: # returns fullpath, is_folder
        # VIDEO_EXTS = (".mp4", ".webm", ".ogg", ".mov", ".mkv")
        # AUDIO_EXTS = (".mp3", ".wav", ".ogg", ".m4a", ".flac")
        # IMAGE_EXTS = (".png", ".jpg", ".jpeg", ".gif", ".svg", ".webp")
        # TEXTLIKE_EXTS = (".md", ".qmd", ".canvas", ".base")
        search_term = urllib.parse.unquote(file); search_term
        rel_path = Path(search_term); rel_path
        fname = rel_path.name
        fldr = rel_path.parent; fldr
        pfname = None
        is_folder = False

        extensions = ExtensionTypes.all()
        if fldr.name == "":
            if fname.endswith(extensions):
                for sub in self.subdirs:
                    pfname = next(sub.rglob(fname), None)
                    if pfname: break
            else:
                mdfname = fname +".md"
                for sub in self.subdirs:
                    pfname = next(sub.rglob(mdfname), None)
                    if pfname: break
        else:
            if fname.endswith(extensions):
                pfname = next((self.vault/fldr).rglob(fname), None)
            else:
                mdfname = fname +".md"
                pfname = next((self.vault/fldr).rglob(mdfname), None)
        content = None
        if pfname: 
            if pfname.name.endswith(".ipynb"): 
                op = JupyterPage.from_file_path(pfname)
                content = op.html
                obsidian_url = op.obsidian_url
                title = op.title
                lockey = op.lockey
                extension = op.file_extension
                response_type=ResponseTypes.html
            elif pfname.name.endswith(ExtensionTypes.AUDIO_EXTS+ExtensionTypes.VIDEO_EXTS+ExtensionTypes.IMAGE_EXTS):
                op = ObsidianPage.from_file_path(pfname)
                content = pfname # pfname.read_bytes()
                obsidian_url = op.obsidian_url
                title = op.title
                lockey = op.lockey
                extension = op.file_extension
                response_type=ResponseTypes.file
            else:
                op = ObsidianPage.from_file_path(pfname)
                content = op.html
                obsidian_url = op.obsidian_url
                title = op.title
                lockey = op.lockey
                extension = op.file_extension
                response_type=ResponseTypes.html
            return {'content': content, 
                    'obsidian_url': obsidian_url, 
                    'is_folder': is_folder, 
                    'title': title,
                    'lockey': lockey,
                    'extension': extension,
                    'response_type':response_type,
                    'backlinks': self.backlinks(op.lockey)}
        else: 
            lsfldr = self.vault/search_term
            if lsfldr.is_dir(): 
                return {'content': self.listing(lsfldr), 
                        'obsidian_url': None, 
                        'is_folder': True, 
                        'title': search_term,
                        'lockey': search_term,
                        'extension': None,
                        'response_type':ResponseTypes.pygen,
                        'backlinks': None} 
            else: 
                return {'content': None, 
                    'obsidian_url': None, 
                    'is_folder': is_folder, 
                    'title': None,
                    'lockey': None,
                    'extension': None,
                    'response_type':ResponseTypes.html,
                    'backlinks': None}

    def backlinks(self, lockey):
        return None

In [ ]:

# vault_path = pathlib.Path(config.OBSIDIAN_VAULT)
# url = http://0.0.0.0:5052/obsidian/open?file=pages/Network.md
fstore = FileStore(config)
dirs = fstore.subdirs ; dirs
list(iter_files(dirs))
# a = fstore.query_file(file='lifeos/Reviews') #['content']
a = fstore.query_file(file='08_ModellingConcepts.png') #['content']
# list(a)
a
# fstore.query_file(file='pages/Network.md')


{'content': Path('/Users/rahul1.saraf/rahuketu/programming/notesobs/assets/08_ModellingConcepts.png'),
 'obsidian_url': 'obsidian://open?vault=notesobs&file=assets/08_ModellingConcepts.png',
 'is_folder': False,
 'title': '08_ModellingConcepts',
 'lockey': 'assets/08_ModellingConcepts.png',
 'extension': 'png',
 'response_type': <ResponseTypes.file: 'file'>,
 'backlinks': None}

In [ ]:
fstore = FileStore(config)
fstore.subdirs[-1]
# list(fstore.listing(fstore.subdirs[-1]))


Path('/Users/rahul1.saraf/rahuketu/programming/notesobs/lifeos/Reviews')

In [ ]:
#| export
class DBStore(NoteStore):

    def __init__(self, config):
        self.config = config
        self.db = Database(config.OBSIDIAN_DB); self.db
        self.vault = config.OBSIDIAN_VAULT; self.vault

    def initdb(self, replace=False) -> Database:
        self.db.create(Node, pk='lockey', replace=replace)
        self.db.create(Links, pk='id', replace=replace, foreign_keys=[
            ('lockey', 'node'),
            # ('linked_lockey', 'node') # Dropping this constraints for efficiency as I want to avoid rereading db twice
        ])
        self.db.create(Properties, pk='id', replace=replace, foreign_keys=[
            ('lockey', 'node')
        ])
        return self.db

    @property
    def subdirs(self):
        return list(self.vault/d['lockey'] for d in self.db['node'].rows_where("is_folder = ?", (1,), select='lockey'))

    def listing(self, folder):
        yield from (d for d in self.db['node'].rows_where("parent = ?", (str(folder),), select='fname, url'))


    def query_file(self, file):
        # VIDEO_EXTS = (".mp4", ".webm", ".ogg", ".mov", ".mkv")
        # AUDIO_EXTS = (".mp3", ".wav", ".ogg", ".m4a", ".flac")
        # IMAGE_EXTS = (".png", ".jpg", ".jpeg", ".gif", ".svg", ".webp")
        # TEXTLIKE_EXTS = (".md", ".qmd", ".canvas", ".base", ".ipynb")
        search_term = urllib.parse.unquote(file); search_term
        rel_path = Path(search_term); rel_path
        fname = rel_path.name
        fldr = rel_path.parent; fldr
        extensions = ExtensionTypes.all()
        data = None
        out = None
        # print(search_term)
        if fname.endswith(ExtensionTypes.TEXTLIKE_EXTS):
            
            data_list = list(self.db['node'].rows_where("fname = ?", (str(fname),), select='fname, url, obsidian_url, is_folder, text, ext, lockey'))
            if data_list:
                # print("1")
                data = data_list[0]
                # print(data)
                op = JupyterPage(data['text'], fpath=fname) if fname.endswith(".ipynb") else ObsidianPage(data['text']) 
                # html = None
                # : html = render_nb(op.text)
                # else: html = op.html
                # print(op.html)
                out = {'content': op.html, 
                       'obsidian_url': data['obsidian_url'], 
                       'is_folder': data['is_folder']>0, 
                       'title': op.title if op.title else pathlib.Path(fname).stem,
                       'lockey': data['lockey'],
                       'extension': data['ext'],
                       'response_type':ResponseTypes.html,
                       'backlinks': self.backlinks(data['lockey'])}
        
        elif fname.endswith(ExtensionTypes.VIDEO_EXTS+ExtensionTypes.AUDIO_EXTS+ExtensionTypes.IMAGE_EXTS):
            
            data_list = list(self.db['node'].rows_where("fname = ?", (str(fname),), select='fname, url, obsidian_url, is_folder, text, blob, ext, lockey'))
            if data_list:
                # print("1")
                data = data_list[0]
                # print(data)
                op = ObsidianPage(data['text'])
                # print(op.html)
                out = {'content': data['blob'], 
                       'obsidian_url': data['obsidian_url'], 
                       'is_folder': data['is_folder']>0, 
                       'title': op.title if op.title else pathlib.Path(fname).stem,
                       'lockey': data['lockey'],
                       'extension': data['ext'],
                       'response_type':ResponseTypes.blob,
                       'backlinks': self.backlinks(data['lockey']) }
        else:
            data_list = list(self.db['node'].rows_where("fname = ?", (str(fname+".md"),), select='fname, url, obsidian_url, is_folder, text, ext, lockey'))
            if data_list:
                data = data_list[0]
                op = ObsidianPage(data['text'])
                out = {'content': op.html, 
                       'obsidian_url': data['obsidian_url'], 
                       'is_folder': data['is_folder']>0, 
                       'title': op.title if op.title else pathlib.Path(fname).stem,
                       'lockey': data['lockey'],
                       'extension': data['ext'],
                       'response_type':ResponseTypes.html,
                       'backlinks': self.backlinks(data['lockey'])}
            else:
                data_list = list(self.db['node'].rows_where("fname = ?", (str(fname),), select='fname, url, obsidian_url, is_folder, text, ext, lockey'))
                if data_list and data_list[0]['is_folder']:
                    data = data_list[0]
                    out = {'content': self.listing(fname),  # Fix this
                           'obsidian_url': data['obsidian_url'], 
                           'is_folder': data['is_folder']>0, 
                           'title': data['fname'],
                           'lockey': data['lockey'],
                           'extension': data['ext'],
                           'response_type':ResponseTypes.pygen,
                           'backlinks': None }
                else: 
                    out = {'content': None, 
                           'obsidian_url': None, 
                           'is_folder': False, 
                           'title': None,
                           'lockey': None,
                           'extension': None,
                           'response_type':ResponseTypes.html,
                           'backlinks': None}
        return out

    def upsert(self, file_path: Path): # Assuming Path is available from pathlib
        # Assuming ObsidianPage, datetime, and timezone are available in the scope.
        # For example, they might be imported at the top of the file as:
        # from pathlib import Path
        # from datetime import datetime, timezone
        # from .mdmanager import ObsidianPage
        
        file_path = pathlib.Path(file_path)
        page = ObsidianPage.from_file_path(file_path)

        if page.lockey is None:
            # This file path is not relative to the vault, which is required for 'lockey'.
            # Log a warning and skip upserting this file.
            print(f"Warning: File path {file_path} is not relative to vault {self.vault}. Skipping upsert.")
            return

        # Check if ObsidianPage's text content is an error message (based on its implementation)
        is_error_text_from_page = page.text and page.text.startswith("[Error:")
        
        node = update_node(page, file_path)
        self.db['node'].upsert(node)
        self.db["links"].delete_where("lockey = ?", (page.lockey,))
        links = get_pagelinks(page, self.vault)
        self.db["links"].insert_all(links)
        self.db["properties"].delete_where("lockey = ?", (page.lockey,))
        self.db["properties"].insert_all(get_properties(page))

    def sync_vault(self, reindex=False):
        fstore = FileStore(self.config)
        dirs = fstore.subdirs; dirs
        for d in dirs: 
            
            fldr_node = update_folder_node(d, fstore.vault)
            # print(d, fldr_node)
            self.db['node'].upsert(fldr_node)
        for f in iter_files(dirs): 
            # check for entry and metadata
            if reindex: self.upsert(f)
            
            lockey = str(f.relative_to(fstore.vault)); lockey
            # print(lockey)
            # print(list(self.db['node'].rows_where("lockey = ?", (lockey,), select='lockey,modified_time,file_size,checksum'))[0])
            infos = list(self.db['node'].rows_where("lockey = ?", (lockey,), select='lockey,modified_time,file_size,checksum'))
            # print(infos)
            if infos:
                info = infos[0]; info
                details = f.stat()
                same_size = details.st_size == info['file_size'] 
                similar_time = details.st_mtime-info['modified_time'] < 1e-6
                if same_size and similar_time: continue
                else:
                    if ObsidianPage.from_file_path(f).checksum == info['checksum']: continue
                    else: 
                        print(f"Syncing {f}")
                        self.upsert(f)
            else:
                self.upsert(f)

    def backlinks(self, lockey):
        print(lockey)
        backlinks = list( Backlink(lockey, bl['lockey'], self.db) for bl in self.db['links'].rows_where("linked_lockey = ?", (str(lockey),), select='lockey'))
        print(backlinks)
        return backlinks

In [ ]:
#| notest
db = Database(config.OBSIDIAN_DB)
subdirs = list(d['fname'] for d in db['node'].rows_where("is_folder = ?", (1,), select='fname')); subdirs
# folder = subdirs[-1]
# folder

['Clippings',
 'Excalidraw',
 'journals',
 'lifeos',
 'templates',
 'assets',
 'pages',
 'Scripts',
 'Downloaded',
 'Reviews']

In [ ]:
#| notest
config.OBSIDIAN_DB
obs = DBStore(config)
obs.initdb()
# obs.subdirs[-1].name
# print(obs.subdirs[-1].name)
# list(obs.listing(str(obs.subdirs[-1].name)))
# fname = 'Clippings/Frequently%20Asked%20Questions%20%28And%20Answers%29%20About%20AI%20Evals%20%E2%80%93%20Hamel%E2%80%99s%20Blog.md'
# fname = 'Clippings'
# # search_term = urllib.parse.unquote('Clippings/Frequently%20Asked%20Questions%20%28And%20Answers%29%20About%20AI%20Evals%20%E2%80%93%20Hamel%E2%80%99s%20Blog.md'); search_term
# # search_term = urllib.parse.unquote('RAG'); search_term
# obs.query_file(fname)

lockey = "pages/RAG.md"
obs.backlinks(lockey)


pages/RAG.md
[Backlink(lockey_page='pages/RAG.md', lockey_backlink='pages/ChatSystem.excalidraw.md', db=<Database <apsw.Connection object "/Users/rahul1.saraf/rahuketu/programming/gitea/memexplatform_obsidian/data/memexplatform_obsidian.db">>), Backlink(lockey_page='pages/RAG.md', lockey_backlink='pages/Product Mindset for RAG.md', db=<Database <apsw.Connection object "/Users/rahul1.saraf/rahuketu/programming/gitea/memexplatform_obsidian/data/memexplatform_obsidian.db">>), Backlink(lockey_page='pages/RAG.md', lockey_backlink='pages/ProductReviews.excalidraw.md', db=<Database <apsw.Connection object "/Users/rahul1.saraf/rahuketu/programming/gitea/memexplatform_obsidian/data/memexplatform_obsidian.db">>)]


[Backlink(lockey_page='pages/RAG.md', lockey_backlink='pages/ChatSystem.excalidraw.md', db=<Database <apsw.Connection object "/Users/rahul1.saraf/rahuketu/programming/gitea/memexplatform_obsidian/data/memexplatform_obsidian.db">>),
 Backlink(lockey_page='pages/RAG.md', lockey_backlink='pages/Product Mindset for RAG.md', db=<Database <apsw.Connection object "/Users/rahul1.saraf/rahuketu/programming/gitea/memexplatform_obsidian/data/memexplatform_obsidian.db">>),
 Backlink(lockey_page='pages/RAG.md', lockey_backlink='pages/ProductReviews.excalidraw.md', db=<Database <apsw.Connection object "/Users/rahul1.saraf/rahuketu/programming/gitea/memexplatform_obsidian/data/memexplatform_obsidian.db">>)]

In [ ]:
#| notest
lockey = 'pages/Checkoslovakia.md'
list(obs.db['node'].rows_where("lockey = ?", (lockey,), 
select='lockey,modified_time,file_size,checksum'))

[]

In [ ]:
#| notest
# obs.sync_vault(reindex=True)
obs.sync_vault()

In [ ]:
#| notest
fstore = FileStore(config)
dirs = fstore.subdirs; dirs
fname = None
for f in iter_files(dirs):
    fname = f
    break

lockey = str(fname.relative_to(config.OBSIDIAN_VAULT)); lockey
info = list(obs.db['node'].rows_where("lockey = ?", (lockey,), select='lockey,modified_time,file_size,checksum'))[0]
info, 
details = fname.stat()
same_size = details.st_size == info['file_size'] 
similar_time = details.st_mtime-info['modified_time'] < 1e-6


In [ ]:
#| notest
obs.db.table_names()

['node', 'links', 'properties', 'sqlite_stat1', 'sqlite_stat4']

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()